# В этом файле когда-то будет парсер

Необходимо спарсить: https://www.ebay.com/sh/research?marketplace=EBAY-US&keywords=Smartphones&dayRange=730&endDate=1684051559220&startDate=1620979559220&categoryId=9355&offset=0&limit=50&tabName=SOLD&tz=Europe%2FMoscow

In [60]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
from random import randint
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.common.by import By
import time
from tqdm import tqdm
import warnings
warnings.filterwarnings('ignore', message='The frame.append method is deprecated and will be removed from pandas in a future version.')

In [72]:
# настраиваем драйвер
options = Options()
options.headless = True  # hide GUI
options.add_argument("--window-size=1920,1080")  # set window size to native GUI size
options.add_argument("start-maximized")  # ensure window is full-screen

# Настраиваем браузер, чтобы он не грузил Java и картинки
chrome_options = webdriver.ChromeOptions()
driver = webdriver.Chrome(options=options, chrome_options=chrome_options)

# Делаем запрос
driver.get("https://www.ebay.com/sh/research?marketplace=EBAY-US&keywords=Smartphones&dayRange=730&endDate=1684051559220&startDate=1620979559220&categoryId=9355&offset=0&limit=50&tabName=SOLD&tz=Europe%2FMoscow")

#Атдыхаем, ждём загрузку
time.sleep(5)

C:\Users\druny\AppData\Local\Temp\ipykernel_15656\60259596.py:3: DeprecationWarning: headless property is deprecated, instead use add_argument('--headless') or add_argument('--headless=new')
  options.headless = True  # hide GUI
C:\Users\druny\AppData\Local\Temp\ipykernel_15656\60259596.py:9: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(options=options, chrome_options=chrome_options)


Тут мы ручками авторизируемся и заходим

Парсим названия и числовые характеристики (Цены, продажи)

In [15]:
df = pd.DataFrame()

In [27]:
def table_parser(soup: BeautifulSoup):
    for product in soup.find_all('tr', class_='research-table-row'):
        title = product.find('span', attrs={'data-item-id': True}).text
        price = product.find('td', class_='research-table-row__item research-table-row__avgSoldPrice').div.find_all('div')[0].text.strip()
        shipping = product.find('td', class_='research-table-row__item research-table-row__avgShippingCost').div.find_all('div')[0].text.strip()
        sales_count = product.find('td', class_='research-table-row__item research-table-row__totalSoldCount').div.find_all('div')[0].text.strip()
        sales_value = product.find('td', class_='research-table-row__item research-table-row__totalSalesValue').div.find_all('div')[0].text.strip()
        try:
            link = product.find('a', class_='research-table-row__link-row-anchor', attrs={'href': True})['href']
        except:
            link = None

        data = {"Title": title, "Price": price, "Shipping_cost": shipping, "Sales_Count": sales_count, "Sales_Value": sales_value, "Link": link}

        global df
        df = df.append(data, ignore_index=True)

In [28]:
for _ in tqdm(range(100)):
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    table_parser(soup)
    next_button = driver.find_element(By.XPATH, '//button[@class="pagination__next"]')
    next_button.click()
    time.sleep(randint(5, 15))
    pass

 55%|█████▌    | 55/100 [10:21<08:28, 11.30s/it]


WebDriverException: Message: disconnected: Unable to receive message from renderer
  (failed to check if window was closed: disconnected: not connected to DevTools)
  (Session info: chrome=113.0.5672.93)
Stacktrace:
Backtrace:
	GetHandleVerifier [0x00988893+48451]
	(No symbol) [0x0091B8A1]
	(No symbol) [0x00825058]
	(No symbol) [0x00818205]
	(No symbol) [0x0081810C]
	(No symbol) [0x00816D46]
	(No symbol) [0x008175CA]
	(No symbol) [0x0082E8A9]
	(No symbol) [0x008291EA]
	(No symbol) [0x00828FC7]
	(No symbol) [0x0087DD16]
	(No symbol) [0x0086A2BC]
	(No symbol) [0x0087C482]
	(No symbol) [0x0086A0B6]
	(No symbol) [0x00847E08]
	(No symbol) [0x00848F2D]
	GetHandleVerifier [0x00BE8E3A+2540266]
	GetHandleVerifier [0x00C28959+2801161]
	GetHandleVerifier [0x00C2295C+2776588]
	GetHandleVerifier [0x00A12280+612144]
	(No symbol) [0x00924F6C]
	(No symbol) [0x009211D8]
	(No symbol) [0x009212BB]
	(No symbol) [0x00914857]
	BaseThreadInitThunk [0x76F47D59+25]
	RtlInitializeExceptionChain [0x77A1B74B+107]

In [ ]:
df.to_csv("Names.csv")

Парсим характеристики моделей

In [38]:
Links = df.Link.tolist()

In [55]:
while None in Links:
    Links.remove(None)
None in Links

False

In [56]:
len(Links)

1755

In [71]:
characteristics_df = pd.DataFrame()

In [73]:
unavaible = []
for cur_link in tqdm(Links):
        if cur_link!= None:
            try:
                driver.get(cur_link)
                driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
                time.sleep(randint(1, 5))
                html = driver.page_source
                soup2 = BeautifulSoup(html, 'html.parser')
                title_tag = soup2.find('h1', {'class': 'x-item-title__mainTitle'})
                title_text = title_tag.text.strip()
                parent_element = driver.find_element(By.CLASS_NAME, 'ux-layout-section--features')
                characteristics = parent_element.text.split("\n")
                characteristics_data = {"Title": title_text}
                for i in range(len(characteristics)):
                    if i%2==0:
                        name = characteristics[i]
                    else:
                        value = characteristics[i]
                        characteristics_data[name] = value

                characteristics_df = characteristics_df.append(characteristics_data, ignore_index=True)
            except:
                print("Сорян, не полусилось:", cur_link)
                unavaible.append(cur_link)
        pass

characteristics_df.to_csv("characteristics_df")

  1%|          | 20/1755 [02:56<3:02:36,  6.31s/it]

Сорян, не полусилось: https://www.ebay.com/itm/224743929590?nordt=true&orig_cvip=true&rt=nc


  2%|▏         | 31/1755 [04:04<2:48:12,  5.85s/it]

Сорян, не полусилось: https://www.ebay.com/itm/254587833287?nordt=true&orig_cvip=true&rt=nc


  8%|▊         | 142/1755 [19:26<9:16:06, 20.69s/it]

...

Сорян, не полусилось: https://www.ebay.com/itm/183335555955?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/193629460245?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/284426022288?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/285136036696?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/353023276525?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/363712905150?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/385120003212?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/403058490423?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/403300018608?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/403531799127?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/403583492402?nordt=true&orig_cvip

 99%|█████████▊| 1732/1755 [2:39:21<00:00, 211.75it/s]

Сорян, не полусилось: https://www.ebay.com/itm/363590174921?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/393332595744?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/401763854259?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/402622410381?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/403589623396?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/124889985422?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/125018556322?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/125121622176?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/143951723451?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/144334024547?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/144650611511?nordt=true&orig_cvip

100%|██████████| 1755/1755 [2:39:21<00:00,  5.45s/it] 


Сорян, не полусилось: https://www.ebay.com/itm/224398959240?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/255581011033?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/265600401017?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/284104102999?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/284800928063?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/313987410998?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/324700579327?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/325275689582?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/401985990556?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/402750536683?nordt=true&orig_cvip=true&rt=nc
Сорян, не полусилось: https://www.ebay.com/itm/403793359382?nordt=true&orig_cvip

In [74]:
characteristics_df

,Title,Состояние товара,Brand,Model,Storage Capacity,Network,Camera Resolution,Screen Size,Lock Status,RAM,...,"ANT+, Bluetooth Enabled, Dust-Resistant, Facial Recognition, Camera, Front Camera, GPS, FM Radio, Nano SIM, Push to Talk, Touch Screen, Global Ready, Night Mode, NFC Connectivity, Rear Camera, Rugged, Sim Free, Speakerphone, Water-Resistant, Wi-Fi Capable",4G(LTE) 3G(WCDMA) and 2G(GSM),"IP68/IP69k/MIL-STD-810H,CE/FCC/Reach/ROHS",248g,5400mAh,8M FF,12M AF + 2M Macro Camera with flash light,Bluetooth 5.0,Support,2.0W Waterproof speaker (Front placed)
0,Apple iPhone 11 64GB Factory Unlocked 4G LTE S...,Очень хорошее состояние - восстановлен,Apple,iPhone 11,64GB,Factory Unlocked,12MP,"6.1""",Network Unlocked,4GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,Apple iPhone XR 64 ГБ заводской разблокированн...,Очень хорошее состояние - восстановлен,Apple,iPhone XR,64GB,Factory Unlocked,12MP,"6.1""",Network Unlocked,3GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,Apple iPhone 11 64 ГБ разблокированный смартфо...,Очень хорошее состояние - восстановлен,Apple,Apple iPhone 11,64 GB,Unlocked,12.0 MP,6.1 in,Factory Unlocked,4 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,Apple iPhone X 64 ГБ заводской разблокированны...,Очень хорошее состояние - восстановлен,Apple,Apple iPhone X,64GB,Factory Unlocked,12.0 MP,5.8 in,Factory Unlocked,3 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,Apple iPhone X 256 ГБ разблокированный смартфо...,Очень хорошее состояние - восстановлен,Apple,Apple iPhone X,256GB,Factory Unlocked,12.0 MP,5.8 in,Factory Unlocked,3 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1085,"Apple iPhone 8 GSM, разблокированный, 64 ГБ, 1...",Хорошее состояние - восстановлен,Apple,Apple iPhone 8,NaN,"AT&T, T-Mobile",1334 x 750 pixels(12.0MP),4.7 in,Network Unlocked,2 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1086,🌟 🌟 совершенно новый Samsung Galaxy S9+ Plus -...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1087,"Смартфон AGM H3 4G тонкий прочный 5,7 дюйма но...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,Lock Status,Certification,Weight,Battery,Chipset Model,Back Camera(Pixel),Processor,NFC,Contract,Acceleration Sensor
1088,Apple iPhone 8 64 ГБ серебро Verizon T-Mobile ...,Хорошее состояние - восстановлен,Apple,Apple iPhone 8,64 GB,Unlocked,12.0 MP,4.7 in,Network Unlocked,2 GB,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [75]:
characteristics_df.to_csv("characteristics_df.csv")